In [1]:
import numpy as np
from os import path, makedirs

from brainpipe.system import study
from brainpipe.feature import TF
from brainpipe.visual import *

from itertools import product

Create single trial TFR for High frequencies (30-120Hz)

In [25]:
#step of 5Hz 30-120Hz width 400ms with 10ms time step
sf = 512
f = (30,120,10,5)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
#f = (2,29,1,1)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
baseline = [768,1024] # 500ms before selected window
width, step = 205, 25 #Time vector (sample) windows 400ms and step of 10ms 17 freqs
#width, step = 358, 25 #Time vector (sample) windows 400ms and step of 10ms 27 freqs
win_sel = [-1,2] # In seconds (start RSA analyses from 0s)

In [26]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/{}/')
data_file = path.join(path_data, '{}_odor_{}_common_renamed_OFC_HC.npz')
path2save = path.join (st.path, 'feature/1_TF_{}_ST_by_cond/')
savename = path.join(path2save, 'TF_ST_{}_{}_30-120.npz')

subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
conds = ['high','low']
reps = ['Encoding_EpiPerf_LowHigh/','Retrieval_EpiPerf_LowHigh/']

for rep,su,cond in product(reps,subjects,conds):
    mat = np.load(data_file.format(rep,su,cond),allow_pickle=True)
    channels, xyz, labels = mat['channels'], mat['xyz'], mat['labels']
    data = mat['x']
    n_elecs, npts, ntrials = data.shape

    kwargs = {} # Define an empty dictionnary to save all power parameters
    kwargs['f'] = f
    kwargs['time_width'], kwargs['time_step'] = width, step
    kwargs['channels'], kwargs['labels'], kwargs['xyz_sel'] = channels, labels, xyz
    time = (np.arange(npts)-(512*3))/sf

    tfObj = TF(sf, npts, f=f, time=time, width=width, step=step,
               baseline=baseline,norm=2)

    TF_by_trial = np.array([])
    for trial in range(ntrials):
        x = data[:,:,trial][:,:,np.newaxis]
        #print('x shape', x.shape)
        x_tf = tfObj.get(x)[0][:,:,:,np.newaxis]
        #print('TF one trial shape',x_tf.shape)
        TF_by_trial = np.concatenate((TF_by_trial,x_tf),axis=-1) if np.size(TF_by_trial) else x_tf
    print('TF_all_trials shape',TF_by_trial.shape)
                
    #select only relevant timepoints
    timebin = np.array(tfObj.xvec)
    sl = slice(np.argmin(np.abs(timebin-win_sel[0])), np.argmin(np.abs(timebin-win_sel[1])))
    kwargs['TF'] = TF_by_trial[:,:,sl,:]
    kwargs['time'] = timebin[sl]
    print('check selection',kwargs['TF'].shape,kwargs['time'].shape)
    
    if not path.exists(path2save.format(rep[0])):
        makedirs(path2save.format(rep[0]))
    np.savez(savename.format(rep[0],su,cond), **kwargs)
    del kwargs['TF']
    del kwargs, x, n_elecs, ntrials, npts
    del data

-> Olfacto loaded
TF_all_trials shape (17, 36, 136, 21)
check selection (17, 36, 61, 21) (61,)
TF_all_trials shape (17, 36, 136, 34)
check selection (17, 36, 61, 34) (61,)
TF_all_trials shape (17, 57, 136, 15)
check selection (17, 57, 61, 15) (61,)
TF_all_trials shape (17, 57, 136, 6)
check selection (17, 57, 61, 6) (61,)
TF_all_trials shape (17, 48, 136, 11)
check selection (17, 48, 61, 11) (61,)
TF_all_trials shape (17, 48, 136, 11)
check selection (17, 48, 61, 11) (61,)
TF_all_trials shape (17, 62, 136, 9)
check selection (17, 62, 61, 9) (61,)
TF_all_trials shape (17, 62, 136, 31)
check selection (17, 62, 61, 31) (61,)
TF_all_trials shape (17, 45, 136, 14)
check selection (17, 45, 61, 14) (61,)
TF_all_trials shape (17, 45, 136, 17)
check selection (17, 45, 61, 17) (61,)
TF_all_trials shape (17, 30, 136, 11)
check selection (17, 30, 61, 11) (61,)
TF_all_trials shape (17, 30, 136, 14)
check selection (17, 30, 61, 14) (61,)
TF_all_trials shape (17, 36, 136, 12)
check selection (17, 36,

Plot TFR for RSA (High and Low conditions)

In [ ]:
st = study('Olfacto')
path_TF = path.join (st.path, 'feature/1_TF_{}_ST_by_cond/')
TF_name = path.join(path2save, 'TF_ST_{}_{}_2-120.npz')
path2save = path.join(path_TF, 'TFR_maps/')
savename = path.join(path2save, 'TF_average_{}_{}_elec({})_{}.png') #su, lab, elec, cond

subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
conds =['high','low']
phases = ['E','R']

for phase,su in product(phases,subjects):
    
    #load and concatenate TFR High and Low to plot (+info)
    mat = np.load(TF_name.format(phase,su,conds[0]),allow_pickle=True)
    chans, labels = mat['channels'], mat['labels']
    time, tf_low = mat['time'], mat['TF'][np.newaxis]
    tf_high = np.load(tf_high_name.format(phase,su,cond))['TF'][np.newaxis]

    TF_conds = np.concatenate((tf_high,tf_low),axis=0)
    _,nfreqs,nelecs,nsamples,ntrials = TF_conds.shape
    print('TF final shape',np.concatenate((tf_high,tf_low),axis=0).shape)
    
    for elec in range(nelecs):
        
        tf_plot = TF_conds[:,:,elec,:,:]
        lab = labels[elec]
        
        #Plots and save figures of TFR
        title = 'TF Low vs High {} {} // elec({})'.format(su,lab,str(elec))
        fig = plt.figure(elec, figsize=(10, 5))
        fig, allax = tfObj.plot2D(fig, tf_plot,time, cmap='viridis',
             yvec=tfObj.yvec, xlabel='Time (ms)', vmin=-150, vmax=150, ycb=-20,
             ylabel='Frequency (hz)', figtitle=title, title=['Low','High'],
             cblabel='Power modulations (%)', pltype='imshow', resample=(0.5, 0.1),
            sharex=False, sharey=False, subdim=(1,3), subspace={'top':0.8})
        for k in allax:
            addLines(k, vLines=lines, vColor=['firebrick']*3, vWidth=[2]*3)
        #Save all your plots
        fname = (path2save+su+'_Elec('+str(elec)+')_'+channel+'_'+label+'_Low_High.pdf')
        fname2 = (path2save+su+'_Elec('+str(elec)+')_'+channel+'_'+label+'_Low_High.png')
        fig.savefig(fname, dpi=300, bbox_inches='tight')
        fig.savefig(fname2, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()

Create single_trial TFR for low frequencies (1-29Hz)

In [2]:
#step of 1Hz 1-29Hz and 5 cycles in time
sf = 512
n_cycles = 5
baseline = [768,1024] # 500ms before selected window
step = 5 #Time vector (sample) windows 400ms and step of 10ms
win_sel = [-1,2] # In seconds (start RSA analyses from 0s)

In [4]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/{}/')
data_file = path.join(path_data, '{}_odor_{}_common_renamed_OFC_HC.npz')
path2save = path.join (st.path, 'feature/1_TF_{}_ST_by_cond/')
savename = path.join(path2save, 'TF_ST_{}_{}_2-29.npz')

subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
conds = ['high','low']
reps = ['Encoding_EpiPerf_LowHigh/','Retrieval_EpiPerf_LowHigh/']
freq_ranges = np.arange(2,30,1)

for rep,su,cond in product(reps,subjects,conds):
    mat = np.load(data_file.format(rep,su,cond),allow_pickle=True)
    channels, xyz, labels = mat['channels'], mat['xyz'], mat['labels']
    data = mat['x']
    n_elecs, npts, ntrials = data.shape
    
    kwargs = {} # Define an empty dictionnary to save all power parameters
    kwargs['channels'], kwargs['labels'], kwargs['xyz'] = channels, labels, xyz
    time = (np.arange(npts)-(512*3))/sf
    
    TF_by_freq = np.array([])
    for freq in freq_ranges:
        width = np.int(((1/freq)*n_cycles)*sf)
        
        tfObj = TF(sf, npts, f=[(freq,freq)], time=time, width=width, step=step,
               baseline=baseline,norm=2)
        timebin = np.array(tfObj.xvec)
        sl = slice(np.argmin(np.abs(timebin-win_sel[0])), np.argmin(np.abs(timebin-win_sel[1])))

        TF_by_trial = np.array([])
        for trial in range(ntrials):
            x = data[:,:,trial][:,:,np.newaxis]
            #print('x shape', x.shape)
            x_tf = tfObj.get(x)[0][:,:,:,np.newaxis]
            print(x_tf.shape)
            0/0
            xtf_sel = x_tf[:,:,sl,:]
            if xtf_sel.shape[2] > 307:
                xtf_sel = xtf_sel[:,:,:-1,:]
            TF_by_trial = np.concatenate((TF_by_trial,xtf_sel),axis=-1) if np.size(TF_by_trial) else xtf_sel
        print('TF_all_trials shape',TF_by_trial.shape)
        TF_by_freq = np.concatenate((TF_by_freq,TF_by_trial),axis=0) if np.size(TF_by_freq) else TF_by_trial
                
    #select only relevant timepoints
    kwargs['TF'] = TF_by_freq
    kwargs['time'] = timebin[sl]
    print('check selection',kwargs['TF'].shape,kwargs['time'].shape)
    
    if not path.exists(path2save.format(rep[0])):
        makedirs(path2save.format(rep[0]))
    np.savez(savename.format(rep[0],su,cond), **kwargs)
    del kwargs['TF']
    del kwargs, x, n_elecs, ntrials, npts
    del data

-> Olfacto loaded
(1, 36, 461, 1)


ZeroDivisionError: division by zero

concatenate low and high frequencies

In [29]:
st = study('Olfacto')
path2save = path.join (st.path, 'feature/1_TF_{}_ST_by_cond/')
tf_low_name = path.join(path2save, 'TF_ST_{}_{}_2-29.npz')
tf_high_name = path.join(path2save, 'TF_ST_{}_{}_30-120.npz')
savename = path.join(path2save, 'TF_ST_{}_{}_2-120.npz')

subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
conds =['high','low']
phases = ['E','R']

for phase,cond,su in product(phases,conds,subjects):
    mat_low = np.load(tf_low_name.format(phase,su,cond),allow_pickle=True)
    #print(mat_low.files)
    
    kwargs = {}
    kwargs['channels'], kwargs['labels'] = mat_low['channels'], mat_low['labels']
    kwargs['xyz'], kwargs['time'] = mat_low['xyz_sel'], mat_low['time']
    
    tf_low = mat_low['TF']
    tf_high = np.load(tf_high_name.format(phase,su,cond))['TF']

    kwargs['TF'] = np.concatenate((tf_high,tf_low),axis=0)
    print('TF final shape',np.concatenate((tf_high,tf_low),axis=0).shape)
    np.savez(savename.format(phase,su,cond), **kwargs)

-> Olfacto loaded
(27, 36, 61, 21)
TF final shape (44, 36, 61, 21)
(27, 57, 61, 15)
TF final shape (44, 57, 61, 15)
(27, 48, 61, 11)
TF final shape (44, 48, 61, 11)
(27, 62, 61, 9)
TF final shape (44, 62, 61, 9)
(27, 45, 61, 14)
TF final shape (44, 45, 61, 14)
(27, 30, 61, 11)
TF final shape (44, 30, 61, 11)
(27, 36, 61, 34)
TF final shape (44, 36, 61, 34)
(27, 57, 61, 6)
TF final shape (44, 57, 61, 6)
(27, 48, 61, 11)
TF final shape (44, 48, 61, 11)
(27, 62, 61, 31)
TF final shape (44, 62, 61, 31)
(27, 45, 61, 17)
TF final shape (44, 45, 61, 17)
(27, 30, 61, 14)
TF final shape (44, 30, 61, 14)
(27, 36, 61, 12)
TF final shape (44, 36, 61, 12)
(27, 57, 61, 12)
TF final shape (44, 57, 61, 12)
(27, 48, 61, 12)
TF final shape (44, 48, 61, 12)
(27, 62, 61, 9)
TF final shape (44, 62, 61, 9)
(27, 45, 61, 12)
TF final shape (44, 45, 61, 12)
(27, 30, 61, 16)
TF final shape (44, 30, 61, 16)
(27, 36, 61, 12)
TF final shape (44, 36, 61, 12)
(27, 57, 61, 12)
TF final shape (44, 57, 61, 12)
(27, 48,